### Load libraries & config

In [1]:
from datetime import date, datetime
import girder_client as gc
import json
import os
from urllib.parse import quote
import random
import re
import string
import tzlocal
from urllib.request import urlopen

with open("config.json", "r") as fp:
    config=json.load(fp)

### Connect to Girder

In [2]:
girder_connection = gc.GirderClient(
    apiUrl="{}/api/v1/".format(
        config["girder-production"]["host"]
    )
)
girder_connection.authenticate(
    username=config["girder-production"]["user"],
    password=config["girder-production"]["password"]
)

In [3]:
activity_sets = girder_connection.get(
    "folder?parentType=collection&parentId={}".format(
        girder_connection.get(
            "collection?name=Volumes"
        )[0]["_id"]
    )
)

### Authorize Viewers to View their Users as Authorized in Admin Panel

In [4]:
for a in activity_sets:
    for v in a['meta']['members']['viewers']:
        for u in a['meta']['members']['viewers'][v]:
            try: 
                access = girder_connection.get("user/{}/access".format(u))
            except:
                access = {'users': []}
            if (v not in [user['id'] for user in access['users']]):
                access['users'].append({'id': v, 'level': 0})
                girder_connection.put("user/{}/access".format(u),
                    data={"access":json.dumps(access)}
                )
            try:
                responses = girder_connection.get(
                    "folder?parentType=user&parentId={}&name=Responses".format(
                        u
                    )
                )
                for responses_folder in responses:
                    try:
                        activity_set_responses = girder_connection.get(
                            "folder?parentType=folder&parentId={}&name={}".format(
                                responses_folder['_id'],
                                a['name']
                            )
                        )
                        for activity_set_response_folder in activity_set_responses: 
                            access = girder_connection.get(
                                "folder/{}/access".format(
                                    activity_set_response_folder['_id']
                                )
                            )
                            if v not in [uid["id"] for uid in access['users']]:
                                access['users'].append({'id': v, 'level': 0})
                                girder_connection.put(
                                    'folder/{}/access'.format(
                                        activity_set_response_folder['_id']
                                    ),
                                    data={
                                        "access":json.dumps(access),
                                        "recurse":True
                                    },
                                )
                    except:
                        ema_parent_responses = None
            except:
                responses = None

### Authorize Managers to View Users as Authorized in Admin Panel

In [5]:
for a in activity_sets:
    for u in a['meta']['members']['users']:
        try:
            access = girder_connection.get(
                'user/{}/access'.format(u)
            )
            for m in a['meta']['members']['managers']:
                if m not in [uid['id'] for uid in access['users']]:
                    access['users'].append({'id': m, 'level': 0})
            girder_connection.put(
                'user/{}/access'.format(u),
                data={"access":json.dumps(access)}
            )
        except:
            access = {}

### Authorize Healthy Brain Network Data Access Group Data Access to All **EMA: Parent** Activity Data

In [6]:
for user in [a for a in activity_sets if a['name'] == "EMA: Parent"][0]['meta']['members']['users']:
    try:
        responses = girder_connection.get(
            "folder?parentType=user&parentId={}&name=Responses".format(
                user
            )
        )
        for responses_folder in responses:
            try:
                ema_parent_responses = girder_connection.get(
                    "folder?parentType=folder&parentId={}&name=EMA%3A%20Parent".format(
                        responses_folder['_id']
                    )
                )
                for ema_parent_response_folder in ema_parent_responses: 
                    access = girder_connection.get(
                        "folder/{}/access".format(
                            ema_parent_response_folder['_id']
                        )
                    )
                    if "5bab8fc80a76ac049becd5ec" not in [gid["id"] for gid in access['groups']]:
                        access['groups'].append({'id': '5bab8fc80a76ac049becd5ec', 'level': 0})
                        girder_connection.put(
                            'folder/{}/access'.format(
                                ema_parent_response_folder['_id']
                            ),
                            data={
                                "access":json.dumps(access),
                                "recurse":True
                            },
                        )
            except:
                ema_parent_responses = None
    except:
        responses = None

### Consolidate duplicated permissions

In [7]:
for u in girder_connection.get('user'):
    access = girder_connection.get(
        'user/{}/access'.format(u['_id'])
    )
    new_access = {'groups': access['groups'], 'users': []}
    for au in access['users']:
        if au['level'] < 2:
            au['level'] = 0
        if au['id'] not in [uid['id'] for uid in new_access['users']]:
            new_access['users'].append(au)
    girder_connection.put(
        'user/{}/access'.format(u["_id"]),
        data={"access":json.dumps(new_access)}
    )

### Purge non-existent users from admin panel

In [ ]:
all_assignees = list({
    *{
        e for a in
        activity_sets for e in
        a['meta']['members']['editors']
    },
    *{
        m for a in
        activity_sets for m in
        a['meta']['members']['managers']
    },
    *{
        u for a in
        activity_sets for u in
        a['meta']['members']['users']
    },
    *{
        v for a in
        activity_sets for v in
        a['meta']['members']['viewers']
    },
    *{
        u for a in
        activity_sets for v in
        a['meta']['members']['viewers'] for u in
        a['meta']['members']['viewers'][v] 
    }
})
purge_list = []
for assignee in all_assignees:
    try:
        girder_connection.getUser(assignee)
    except:
        purge_list.append(assignee)
for a in activity_sets:
    original_members = a['meta']['members'].copy()
    for role in ['editors', 'users', 'managers']:
        a['meta']['members'][role] = list(set(a['meta']['members'][role]) - set(purge_list))
    for purgee in purge_list:
        a['meta']['members']['viewers'].pop(purgee, None)
    for v in a['meta']['members']['viewers']:
        a['meta']['members']['viewers'][v] = list(set(a['meta']['members']['viewers'][v]) - set(purge_list))
    if(original_members != a['meta']['members']):
        girder_connection.put(
            'folder/{}'.format(
                a['_id']
            ),
            data={
                'metadata':json.dumps(
                    a['meta']
                )
            }
        )

In [ ]:
for manager in [a for a in activity_sets if a['name'] == "EMA: Parent"][0]['meta']['members']['managers']:
    u = girder_connection.getUser(manager)
    print("{} {}".format(u['firstName'], u['lastName']))

In [8]:
all_assignees = list({
    *{
        e for a in
        activity_sets for e in
        a['meta']['members']['editors']
    },
    *{
        m for a in
        activity_sets for m in
        a['meta']['members']['managers']
    },
    *{
        u for a in
        activity_sets for u in
        a['meta']['members']['users']
    },
    *{
        v for a in
        activity_sets for v in
        a['meta']['members']['viewers']
    },
    *{
        u for a in
        activity_sets for v in
        a['meta']['members']['viewers'] for u in
        a['meta']['members']['viewers'][v] 
    }
})
for assignee in all_assignees:
    if assignee not in [a for a in activity_sets if a['name'] == "EMA: Parent"][0]['meta']['members']['users']:
        u = girder_connection.getUser(assignee)
        print("{} {} {}".format(u['firstName'], u['lastName'], u['_id']))

arno1000 klein 5bb6655d0a76ac049becd672
Top Cat 5c1924c2c25b5a144b269f04
Erik3 Ilyin 5bb86b510a76ac049becd6b2
Paul Mitrani 5c105091c25b5a0ddcdfd3ce
Erik Good1 5b9b2db5e534fe0e917efdd2
Miriam San Lucas 5c2fcfebc25b5a625531c8fd
Admin Admin 5c1411e0c25b5a144b269e63
Kristen Schiavo 5c2d22c4c25b5a6a94d5fff6
Winter School 5c12c060c25b5a7c7a24d95b
Secret Squirrel 5bf5d445336da85b0dfed575
Erik4 Ilyin 5bb86c660a76ac049becd6b3
Erik P 5b634cbd0fc13a0df633adfd
Arno Klein 5bc4e41f01a25f0dd6703f98
arno1001 klein 5bb668700a76ac049becd673
Grace McAlindin 5c2d22dcc25b5a6a94d5fff7
Erik Ilyin 5b7f1ca0e534fe32c5f5db89
Anna MacKay-Brandt 5c180e56c25b5a144b269e66
Madeline Walsh 5c3633afa0e4760df6a033fc
Erik Good 5b9b2cc3e534fe0e917efdd1
Super Admin 5b634cbe0fc13a0df633adfe
arno1002 klein 5bb668bf0a76ac049becd674
Donald Duck 5be5b321336da80de9146950
Arno Klein 5bc4e30401a25f0dd6703f96
admin admin 5c1801c4c25b5a144b269e65
Marcos Vidor 5bf33623336da80de9146a26
Dheeraj Sood 5c30f21fc25b5a625531c94f
